In [1]:
import os
import numpy as np
import pandas as pd
import pylab as pl
import datetime as dt

pd.options.display.max_columns = None
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Clean station information**

This data is gathering from Nov 2018

In [64]:
citibike_station = pd.read_csv('data/station_information.csv')
citibike_station.head(1)

,capacity,external_id,has_kiosk,lat,lon,name,region_id,rental_methods,rental_url,short_name,station_id,eightd_has_key_dispenser,bikes_availability,description,docks_availability,id_num,services_name,schedule_description,service_type
0,37,66db6aae-0aca-11e7-82f6-3863bb44ef7c,True,40.722174,-73.983688,E 2 St & Avenue B,71.0,"['KEY', 'CREDITCARD']",http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,5515.02,301,False,UNLIMITED,Citi Bike Station Valet attendant service avai...,UNLIMITED,c36f6075-0d24-479b-8018-30790d303769,Valet Service,NaN,ATTENDED_SERVICE


In [65]:
citibike_station.columns

Index(['capacity', 'external_id', 'has_kiosk', 'lat', 'lon', 'name',
       'region_id', 'rental_methods', 'rental_url', 'short_name', 'station_id',
       'eightd_has_key_dispenser', 'bikes_availability', 'description',
       'docks_availability', 'id_num', 'services_name', 'schedule_description',
       'service_type'],
      dtype='object')

In [66]:
citibike_station.drop(['external_id', 'has_kiosk', 'region_id', 'rental_methods', 'rental_url',
                       'eightd_has_key_dispenser', 'bikes_availability', 'description',
                       'docks_availability', 'id_num', 'services_name', 'schedule_description',
                       'service_type'], axis=1, inplace=True)

In [67]:
citibike_station.head()

,capacity,lat,lon,name,short_name,station_id
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72


In [6]:
citibike_station.to_csv('clean_data/citibike_station.csv', index=False)

**Combine trip data into station**

Use 2018 trip data.

In [7]:
tripdata = pd.read_csv('Data/201801-citibike-tripdata.csv')

In [8]:
tripdata.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [9]:
tripdata.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,970,2018-01-01 13:50:57.4340,2018-01-01 14:07:08.1860,72,W 52 St & 11 Ave,40.767272,-73.993929,505,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1
1,723,2018-01-01 15:33:30.1820,2018-01-01 15:45:33.3410,72,W 52 St & 11 Ave,40.767272,-73.993929,3255,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1
2,496,2018-01-01 15:39:18.3370,2018-01-01 15:47:35.1720,72,W 52 St & 11 Ave,40.767272,-73.993929,525,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1
3,306,2018-01-01 15:40:13.3720,2018-01-01 15:45:20.1910,72,W 52 St & 11 Ave,40.767272,-73.993929,447,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1
4,306,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72,W 52 St & 11 Ave,40.767272,-73.993929,3356,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1


In [10]:
tripdata.drop(['start station name', 'start station latitude','start station longitude', 
                       'end station name','end station latitude', 'end station longitude', 
                       'usertype'], axis=1, inplace=True)

In [11]:
tripdata.head()

,tripduration,starttime,stoptime,start station id,end station id,bikeid,birth year,gender
0,970,2018-01-01 13:50:57.4340,2018-01-01 14:07:08.1860,72,505,31956,1992,1
1,723,2018-01-01 15:33:30.1820,2018-01-01 15:45:33.3410,72,3255,32536,1969,1
2,496,2018-01-01 15:39:18.3370,2018-01-01 15:47:35.1720,72,525,16069,1956,1
3,306,2018-01-01 15:40:13.3720,2018-01-01 15:45:20.1910,72,447,31781,1974,1
4,306,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72,3356,30319,1992,1


In [12]:
filename = []

for root, dirs, files in os.walk('Data'):  
    #print(files)
    for file in files: 
        if file[0:4] == '2018':
            filename.append(file)

In [13]:
filename

['201801-citibike-tripdata.csv',
 '201802-citibike-tripdata.csv',
 '201803-citibike-tripdata.csv',
 '201804-citibike-tripdata.csv',
 '201805-citibike-tripdata.csv',
 '201806-citibike-tripdata.csv',
 '201807-citibike-tripdata.csv',
 '201808-citibike-tripdata.csv',
 '201809-citibike-tripdata.csv',
 '201810-citibike-tripdata.csv',
 '201811-citibike-tripdata.csv',
 '201812-citibike-tripdata.csv']

In [14]:
tripdata_org = pd.read_csv('Data/' + filename[0])
tripdata_org.drop(['start station name', 'start station latitude','start station longitude', 
               'end station name','end station latitude', 'end station longitude', 
               'usertype'], axis=1, inplace=True)
#tripdata['starttime'] = pd.to_datetime(tripdata['starttime'])
tripdata_org['starttime'] = pd.to_datetime(tripdata_org['starttime'], infer_datetime_format=True)
tripdata_org['stoptime'] = pd.to_datetime(tripdata_org['stoptime'], infer_datetime_format=True)


for item in filename[1:]:
    print(item)
    tripdata = pd.read_csv('Data/' + item)
    tripdata.drop(['start station name', 'start station latitude','start station longitude', 
                   'end station name','end station latitude', 'end station longitude', 
                   'usertype'], axis=1, inplace=True)
    tripdata['starttime'] = pd.to_datetime(tripdata['starttime'], infer_datetime_format=True)
    tripdata['stoptime'] = pd.to_datetime(tripdata['stoptime'], infer_datetime_format=True)
    tripdata_org = pd.concat([tripdata_org, tripdata])

201802-citibike-tripdata.csv
201803-citibike-tripdata.csv
201804-citibike-tripdata.csv
201805-citibike-tripdata.csv
201806-citibike-tripdata.csv
201807-citibike-tripdata.csv
201808-citibike-tripdata.csv
201809-citibike-tripdata.csv
201810-citibike-tripdata.csv
201811-citibike-tripdata.csv
201812-citibike-tripdata.csv


In [15]:
tripdata_org.head()

,tripduration,starttime,stoptime,start station id,end station id,bikeid,birth year,gender
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,505.0,31956,1992,1
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,3255.0,32536,1969,1
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,525.0,16069,1956,1
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,447.0,31781,1974,1
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,3356.0,30319,1992,1


In [30]:
#tripdata_org.to_csv('Clean_Data/tripdata_new.csv', index=False)

Count the start and end trip number per hour for each station.

In [45]:
stations = citibike_station

In [46]:
stations.shape

(818, 6)

In [18]:
tripdata = tripdata_org

In [25]:
tripdata = tripdata.dropna()

In [26]:
tripdata.shape

(17545842, 8)

In [27]:
tripdata = tripdata.reset_index(drop = True)

In [28]:
tripdata['starttime'][1000].hour

18

In [29]:
# tripdata['starttime'] = tripdata['starttime'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
# tripdata['stoptime'] = tripdata['stoptime'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [30]:
tripdata['startHour'] = tripdata.apply(lambda x: x['starttime'].hour, axis=1)

In [35]:
tripdata['endHour'] = tripdata.apply(lambda x: x['stoptime'].hour, axis=1)

In [36]:
hourRange = range(24)

In [39]:
tripdata.head()

,tripduration,starttime,stoptime,start station id,end station id,bikeid,birth year,gender,startHour,endHour
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,505.0,31956,1992,1,13,14
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,3255.0,32536,1969,1,15,15
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,525.0,16069,1956,1,15,15
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,447.0,31781,1974,1,15,15
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,3356.0,30319,1992,1,18,18


In [55]:
#tripstartHour = []
citibike_station.head()

,capacity,lat,lon,name,short_name,station_id
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72


In [68]:
for i in range(24):
    print(i)

    tripstartHour = tripdata[tripdata['startHour'] == i]
    print(tripstartHour.shape)
    tripstartCount = tripstartHour.groupby('start station id').count()
    #print(tripstartCount.reset_index(drop = True))
    tripstartCount = tripstartCount.reset_index()[['start station id', 'tripduration']]
    
    tripendHour = tripdata[tripdata['endHour'] == i]
    print(tripendHour.shape)
    tripendCount = tripendHour.groupby('end station id').count()
    tripendCount = tripendCount.reset_index()[['end station id', 'starttime']]

    citibike_station = citibike_station.merge(tripstartCount, how='left', left_on='station_id', right_on='start station id')
    citibike_station = citibike_station.merge(tripendCount, how='left', left_on='station_id', right_on='end station id')
    
    #print(citibike_station.head())
    citibike_station.drop(['start station id', 'end station id'], axis=1, inplace=True)
    start_str = 'startCount_' + str(i)
    end_str = 'endCount_' + str(i)
    citibike_station = citibike_station.rename(index=str, columns={"tripduration": start_str, "starttime": end_str})

0
(153446, 10)
(174919, 10)
1
(85807, 10)
(97451, 10)
2
(50095, 10)
(56051, 10)
3
(31109, 10)
(34488, 10)
4
(32749, 10)
(31372, 10)
5
(113212, 10)
(92176, 10)
6
(382809, 10)
(327581, 10)
7
(807717, 10)
(694962, 10)
8
(1412613, 10)
(1310167, 10)
9
(1111786, 10)
(1218572, 10)
10
(785925, 10)
(805542, 10)
11
(826415, 10)
(787880, 10)
12
(956407, 10)
(931792, 10)
13
(1000640, 10)
(990156, 10)
14
(1019523, 10)
(1009525, 10)
15
(1072821, 10)
(1055582, 10)
16
(1252623, 10)
(1202087, 10)
17
(1706927, 10)
(1623834, 10)
18
(1625845, 10)
(1700611, 10)
19
(1131125, 10)
(1243245, 10)
20
(770084, 10)
(840404, 10)
21
(543630, 10)
(585633, 10)
22
(404663, 10)
(434969, 10)
23
(267871, 10)
(296843, 10)


In [69]:
citibike_station.head()

,capacity,lat,lon,name,short_name,station_id,startCount_0,endCount_0,startCount_1,endCount_1,startCount_2,endCount_2,startCount_3,endCount_3,startCount_4,endCount_4,startCount_5,endCount_5,startCount_6,endCount_6,startCount_7,endCount_7,startCount_8,endCount_8,startCount_9,endCount_9,startCount_10,endCount_10,startCount_11,endCount_11,startCount_12,endCount_12,startCount_13,endCount_13,startCount_14,endCount_14,startCount_15,endCount_15,startCount_16,endCount_16,startCount_17,endCount_17,startCount_18,endCount_18,startCount_19,endCount_19,startCount_20,endCount_20,startCount_21,endCount_21,startCount_22,endCount_22,startCount_23,endCount_23
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301,384.0,818.0,233.0,413.0,129.0,179.0,92.0,93.0,79.0,69.0,301.0,68.0,1175.0,287.0,2909.0,719.0,5507.0,1164.0,2118.0,949.0,1724.0,1187.0,1923.0,1553.0,2040.0,1809.0,2143.0,2001.0,2069.0,2189.0,2111.0,2589.0,2299.0,3074.0,2561.0,4909.0,3019.0,5817.0,2619.0,4735.0,1875.0,3861.0,1331.0,2763.0,1018.0,2228.0,701.0,1428.0
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445,501.0,1307.0,387.0,685.0,238.0,358.0,129.0,251.0,155.0,113.0,381.0,121.0,1342.0,425.0,4265.0,1100.0,10419.0,1874.0,6121.0,1432.0,4069.0,1758.0,3262.0,1910.0,3569.0,2364.0,3150.0,2807.0,2805.0,3030.0,2885.0,3362.0,2941.0,4302.0,3531.0,6856.0,4964.0,9535.0,3917.0,10358.0,2252.0,7950.0,1479.0,4801.0,1164.0,3341.0,1105.0,2287.0
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487,240.0,789.0,113.0,338.0,56.0,198.0,36.0,152.0,241.0,100.0,737.0,134.0,2872.0,679.0,5207.0,1073.0,7540.0,1637.0,4836.0,1120.0,2369.0,1096.0,2211.0,1733.0,2143.0,1988.0,2236.0,2334.0,2313.0,2782.0,2747.0,3086.0,2828.0,3923.0,3709.0,7181.0,3610.0,5584.0,2819.0,3809.0,1869.0,3493.0,1270.0,2916.0,931.0,2288.0,511.0,1403.0
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504,1101.0,1217.0,559.0,668.0,322.0,481.0,220.0,299.0,219.0,194.0,954.0,315.0,3222.0,1542.0,5290.0,2514.0,6041.0,2683.0,3030.0,2288.0,2590.0,2209.0,2897.0,2436.0,3242.0,3167.0,3251.0,3344.0,3352.0,3530.0,3798.0,4110.0,4140.0,4549.0,4792.0,7473.0,4777.0,8050.0,4307.0,5963.0,3264.0,4116.0,2317.0,3000.0,2145.0,2749.0,1666.0,1780.0
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72,335.0,349.0,240.0,179.0,165.0,78.0,84.0,61.0,117.0,24.0,273.0,66.0,510.0,722.0,1714.0,1131.0,3383.0,2035.0,2969.0,3081.0,1918.0,2272.0,1807.0,1762.0,2118.0,1881.0,2218.0,2038.0,2144.0,2206.0,2281.0,2436.0,2762.0,2935.0,3467.0,3383.0,3496.0,3853.0,2588.0,2950.0,1535.0,1922.0,916.0,1351.0,800.0,1224.0,565.0,653.0


In [70]:
citibike_station.to_csv('Clean_Data/stations_features.csv', index=False)